In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

cleaned_chess_data_path = r'C:\Users\Abrah\Dropbox\PC (2)\Desktop\GitHub Repos\CST499-40_FA22-Capstone-BradleyChess\chess_data\chess_data_cleaned.pkl'
# bradley_q_table_path = r"C:\Users\Abrah\Dropbox\PC (2)\Desktop\GitHub Repos\CST499-40_FA22-Capstone-BradleyChess\Q_Tables\bradley_agent_q_table.pkl"


# kaggle_chess_data = pd.read_pickle(kaggle_chess_data_path, compression = 'zip')
cleaned_chess_data = pd.read_pickle(cleaned_chess_data_path, compression = 'zip')
# bradley_q_table = pd.read_pickle(bradley_q_table_path, compression = 'zip')

In [ ]:
cleaned_chess_data.head(10)

In [ ]:
bradley_q_table.head(20)

In [ ]:
bradley_q_table.shape

In [ ]:
chess_pd_dataframe_file_path_part_1 = r"C:\Users\Abrah\Dropbox\PC (2)\Desktop\Chess_Games_Database\Chess_Games_DB_pd_df_Part_1.pkl"

In [ ]:
chess_data_from_json = pd.read_pickle(chess_pd_dataframe_file_path_part_1, compression = 'zip')

In [ ]:
chess_data_from_json.head(10)

In [ ]:
chess_data_from_json.info(memory_usage='deep')

In [ ]:
games_with_more_than_100_moves = chess_data_from_json[chess_data_from_json['Num Moves'] > 300]

In [ ]:
games_with_more_than_100_moves.head(10)

In [ ]:
games_with_more_than_100_moves.shape

In [ ]:
import pandas as pd
import chess.pgn
import game_settings
import time

def pgn_to_dataframe(input_file_path):
    games_list = []

    with open(input_file_path, 'r', encoding='utf-8') as file:
        while True:
            game = chess.pgn.read_game(file)
            if game is None:
                break

            game_dict = {
                "PlyCount": game.end().board().ply(),
                "Result": game.headers["Result"]
            }

            board = game.board()
            for i, move in enumerate(game.mainline_moves()):
                col_prefix = 'W' if i % 2 == 0 else 'B'
                move_num = i // 2 + 1
                col_name = f"{col_prefix}{move_num}"
                game_dict[col_name] = board.san(move)
                board.push(move)

            games_list.append(game_dict)

    chess_df = pd.DataFrame(games_list)
    chess_df.index = ['Game ' + str(i+1) for i in chess_df.index]
    chess_df = chess_df.fillna('')
    chess_df = chess_df[[c for c in chess_df if c not in ['Result']] + ['Result']]
    chess_df = chess_df[(chess_df['PlyCount'] > 0) & (chess_df['PlyCount'] <= game_settings.max_num_turns_per_player * 2)]

    return chess_df




In [ ]:
start_time = time.time()
df = pgn_to_dataframe(game_settings.chess_pgn_file_path_1)
df.to_pickle(game_settings.chess_pd_dataframe_file_path_part_1, compression='zip')
end_time = time.time()
print('PGN to DataFrame conversion is complete\n')
print(f'It took: {end_time - start_time} seconds')

In [ ]:
import game_settings
import pandas as pd
chess_data = pd.read_pickle(game_settings.chess_pd_dataframe_file_path_part_1, compression = 'zip')

In [ ]:
chess_data.shape